# CS 640 Homework 3 - Planning with Search

In this homework, you will implement a planning algorithm based on search to play a simplified version of the [Snake](https://en.wikipedia.org/wiki/Snake_(video_game_genre)) arcade game.
In this game, you control the movement of a snake and the goal is to eat food on the board.
The snake moves by directing its head and the rest of the body follows the head.
The goal is to move the head to the food once.
Like in the arcade game, moving the snake into itself results in a failure.


## Instructions

An implementation of this game is below which provides a class to represent a current state of the game.
You should not need to change this class.

1. Implement the `plan` function further down to pick a minimum sequence of moves for the snake to get the food and achieve the goal.
2. Run the five tests at the bottom of this notebook to demonstrate your `plan` function.

Note that these tests check if your plan picks legal moves and achieves the goal.
These tests will not check if your plan used the minimum number of moves for each starting state, but that will still be part of your grade.

**Submit your updated notebook in Gradescope when you are done.**

## Baby Snake Game Implementation

In [12]:
DIRECTIONS = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

class BabySnakeState(object):
    def __init__(self, food_pos, snake_pos):
        self.food_pos = tuple(food_pos)
        self.snake_pos = tuple(map(tuple, snake_pos))

        assert len(self.food_pos) == 2

        assert len(self.snake_pos) > 0
        assert all(len(p) == 2 for p in self.snake_pos)

    def __str__(self):
        lines = [['.'] * 5 for _ in range(5)]

        lines[self.food_pos[0]][self.food_pos[1]] = 'F'

        snake_head = self.snake_pos[0]
        snake_body = self.snake_pos[1:]

        lines[snake_head[0]][snake_head[1]] = 'S'
        for (i, body_part) in enumerate(snake_body):
            lines[body_part[0]][body_part[1]] = chr(ord('0') + i % 10)

        return '\n'.join([''.join(line) for line in lines])

    def is_goal(self):
        return self.food_pos in self.snake_pos

    def get_successors(self):
        successors = []
        for direction in DIRECTIONS:
            new_state = self.move(direction)
            if new_state is not None:
                successors.append(new_state)
        return successors

    def move(self, direction):
        assert direction in DIRECTIONS

        new_head = (self.snake_pos[0][0] + DIRECTIONS[direction][0],
                    self.snake_pos[0][1] + DIRECTIONS[direction][1])
        new_head = tuple(p % 5 for p in new_head)

        if new_head in self.snake_pos[:-1]:
            # snake collided with self
            return None

        new_snake_pos = (new_head,) + self.snake_pos[:-1]

        return BabySnakeState(self.food_pos, new_snake_pos)

test_state = BabySnakeState([3, 3], [[1, 1], [1, 2]])
print("TEST STATE")
print(test_state)
for (i, s) in enumerate(test_state.get_successors()):
    print("SUCCESSOR", i)
    print(s)

TEST STATE
.....
.S0..
.....
...F.
.....
SUCCESSOR 0
.S...
.0...
.....
...F.
.....
SUCCESSOR 1
.....
.0...
.S...
...F.
.....
SUCCESSOR 2
.....
S0...
.....
...F.
.....
SUCCESSOR 3
.....
.0S..
.....
...F.
.....


## Random Walk through State Space

In [2]:
import random

In [3]:
random.seed(640)

s = test_state
for i in range(100):
    print("RANDOM WALK", i)
    print(s)
    if s.is_goal():
        break

    s = random.choice(s.get_successors())

RANDOM WALK 0
.....
.S0..
.....
...F.
.....
RANDOM WALK 1
.S...
.0...
.....
...F.
.....
RANDOM WALK 2
.0S..
.....
.....
...F.
.....
RANDOM WALK 3
..0..
..S..
.....
...F.
.....
RANDOM WALK 4
..S..
..0..
.....
...F.
.....
RANDOM WALK 5
.S0..
.....
.....
...F.
.....
RANDOM WALK 6
.0...
.S...
.....
...F.
.....
RANDOM WALK 7
.....
.0...
.S...
...F.
.....
RANDOM WALK 8
.....
.....
S0...
...F.
.....
RANDOM WALK 9
.....
.....
0S...
...F.
.....
RANDOM WALK 10
.....
.....
.0S..
...F.
.....
RANDOM WALK 11
.....
.....
..0S.
...F.
.....
RANDOM WALK 12
.....
.....
..S0.
...F.
.....
RANDOM WALK 13
.....
.....
..0S.
...F.
.....
RANDOM WALK 14
.....
.....
...0.
...S.
.....


## Write a Planning Function

The goal of this little snake game is to control the snake's movement so that the food is in the snake.
Modify the `plan` function below so that the snake gets the food in the minimum number of moves.

In [13]:
from collections import deque

def plan(state):
    moves = list(DIRECTIONS.keys())  # ["up", "down", "left", "right"]

    queue = deque([(state, [])])
    visited = set([state.snake_pos])  # store snake positions, hashable

    while queue:
        current_state, path = queue.popleft()

        # Goal check
        if current_state.is_goal():
            return path

        # Explore neighbors
        for move in moves:
            next_state = current_state.move(move)
            if next_state and next_state.snake_pos not in visited:
                visited.add(next_state.snake_pos)
                queue.append((next_state, path + [move]))

    return []  # no solution found



In [14]:
plan(test_state)

['down', 'down', 'right', 'right']

### Tests

Run the following tests to check the results of your `plan` function.
These tests will check if your plan picks legal moves and stops after getting the food.
They will not check if your plan has the minimum number of moves, but you will be graded on that.

In [15]:
def test(food_pos, snake_pos):
    state = BabySnakeState(food_pos, snake_pos)
    print("INITIAL STATE")
    print(state)
    print("")

    p = plan(state)
    for m in p:
        # make sure not moving after reaching goal
        assert state.is_goal() == False

        state = state.move(m)
        # make sure move was legal
        assert state is not None

        print("NEXT MOVE:", m)
        print(state)
        print("")

    # make sure stopped at goal
    assert state.is_goal()

    print(f"Goal achieved with {len(p)} moves.")

#### Test 1

In [16]:
test(food_pos=(0, 0), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
F....
S012.
.....
.....
.....

NEXT MOVE: up
S....
012..
.....
.....
.....

Goal achieved with 1 moves.


#### Test 2

In [17]:
test(food_pos=(4, 2), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
.....
S012.
.....
.....
..F..

NEXT MOVE: up
S....
012..
.....
.....
..F..

NEXT MOVE: up
0....
12...
.....
.....
S.F..

NEXT MOVE: right
1....
2....
.....
.....
0SF..

NEXT MOVE: right
2....
.....
.....
.....
10S..

Goal achieved with 4 moves.


#### Test 3

In [18]:
test(food_pos=(1, 4), snake_pos=[(1, 0), (1, 1), (1, 2), (1, 3)])

INITIAL STATE
.....
S012F
.....
.....
.....

NEXT MOVE: left
.....
012.S
.....
.....
.....

Goal achieved with 1 moves.


#### Test 4

In [19]:
test(food_pos=(2, 2), snake_pos=[(3,4), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3)])

INITIAL STATE
.....
.456.
.3F7.
.210S
.....

NEXT MOVE: up
.....
.567.
.4F.S
.3210
.....

NEXT MOVE: left
.....
.67..
.5FS0
.4321
.....

NEXT MOVE: left
.....
.7...
.6S01
.5432
.....

Goal achieved with 3 moves.


#### Test 5

In [20]:
test(food_pos=(2,2), snake_pos=[(4, 1), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 2), (4, 2), (4, 3)])

INITIAL STATE
.....
.234.
.1F5.
.076.
.S89.

NEXT MOVE: down
.S...
.345.
.2F6.
.187.
.09..

NEXT MOVE: right
.0S..
.456.
.3F7.
.298.
.1...

NEXT MOVE: up
.10..
.567.
.4F8.
.3.9.
.2S..

NEXT MOVE: up
.21..
.678.
.5F9.
.4S..
.30..

NEXT MOVE: up
.32..
.789.
.6S..
.50..
.41..

Goal achieved with 5 moves.
